In [1]:
import pandas as pd
import ast
from sklearn.model_selection import KFold
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import numpy as np

In [2]:
def word2features(sent, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    
    if i > 0:
        prev_word = sent[i-1]
        features.update({
            '-1:word.lower()': prev_word.lower(),
            '-1:word.istitle()': prev_word.istitle(),
            '-1:word.isupper()': prev_word.isupper(),
        })
    else:
        features['BOS'] = True  
    
    if i < len(sent) - 1:
        next_word = sent[i+1]
        features.update({
            '+1:word.lower()': next_word.lower(),
            '+1:word.istitle()': next_word.istitle(),
            '+1:word.isupper()': next_word.isupper(),
        })
    else:
        features['EOS'] = True  
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [3]:
df = pd.read_csv("PET_Customized.csv")

df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['ner_tags'] = df['ner_tags'].apply(ast.literal_eval)

X = [sent2features(tokens) for tokens in df['tokens']]
y = [tags for tags in df['ner_tags']]

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

target_labels = [
    "B-Actor", "I-Actor",
    "B-Activity", "I-Activity",
    "B-Activity Data", "I-Activity Data"
]

results = []
fold_num = 1

In [5]:
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test  = [X[i] for i in test_index]
    y_train = [y[i] for i in train_index]
    y_test  = [y[i] for i in test_index]
    
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    y_pred = crf.predict(X_test)
    
    precision = metrics.flat_precision_score(y_test, y_pred, labels=target_labels, average='weighted')
    recall = metrics.flat_recall_score(y_test, y_pred, labels=target_labels, average='weighted')
    f1 = metrics.flat_f1_score(y_test, y_pred, labels=target_labels, average='weighted')
    
    results.append({
        'Fold': fold_num,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })
    fold_num += 1

In [6]:
results_df = pd.DataFrame(results)

average_metrics = {
    'Fold': 'Average',
    'Precision': results_df['Precision'].mean(),
    'Recall': results_df['Recall'].mean(),
    'F1 Score': results_df['F1 Score'].mean()
}

results_df = pd.concat([results_df, pd.DataFrame([average_metrics])], ignore_index=True)

print(results_df)

      Fold  Precision    Recall  F1 Score
0        1   0.782731  0.696746  0.735288
1        2   0.800582  0.757685  0.776582
2        3   0.764996  0.669231  0.710201
3        4   0.715690  0.657194  0.681713
4        5   0.783208  0.726481  0.753156
5  Average   0.769441  0.701467  0.731388
